In [4]:
import pandas as pd
import numpy as np
application_train_df = pd.read_csv('Datasets/application_train.csv')
bureau_df = pd.read_csv('Datasets/bureau.csv')



In [ ]:
application_train_df['FLAG_OWN_CAR'] = application_train_df['FLAG_OWN_CAR'].map({'Y': 1, 'N': 0})
application_train_df['FLAG_OWN_REALTY'] = application_train_df['FLAG_OWN_REALTY'].map({'Y': 1, 'N': 0})
application_train_df['NAME_CONTRACT_TYPE'] = application_train_df['NAME_CONTRACT_TYPE'].map({'Cash loans': 0, 'Revolving loans': 1})
application_train_df['CODE_GENDER'] = application_train_df['CODE_GENDER'].map({'M': 0, 'F': 1, 'XNA': 2})
application_train_df['NAME_TYPE_SUITE'] = application_train_df['NAME_TYPE_SUITE'].map({'Unaccompanied': 0, 'Family': 1, 'Spouse, partner': 2,
                                                   'Children': 3, 'Other_A': 4, 'Other_B': 5, np.nan: 6})
application_train_df['NAME_INCOME_TYPE'] = application_train_df['NAME_INCOME_TYPE'].map({'Working': 0, 'State servant': 1, 'Commercial associate': 2,
                                                     'Pensioner': 3, 'Unemployed': 4, 'Student': 5, 'Businessman': 6, 'Maternity leave': 7})
application_train_df['NAME_EDUCATION_TYPE'] = application_train_df['NAME_EDUCATION_TYPE'].map({'Secondary / secondary special': 0, 'Higher education': 1,
                                                           'Incomplete higher': 2, 'Lower secondary': 3, 'Academic degree': 4})
application_train_df['NAME_FAMILY_STATUS'] = application_train_df['NAME_FAMILY_STATUS'].map({'Married': 0, 'Single / not married': 1, 'Civil marriage': 2,
                                                         'Separated': 3, 'Widow': 4})             
application_train_df['NAME_HOUSING_TYPE'] = application_train_df['NAME_HOUSING_TYPE'].map({'House / apartment': 0, 'With parents': 1, 'Municipal apartment': 2,
                                                       'Rented apartment': 3, 'Office apartment': 4, 'Co-op apartment': 5})
application_train_df['OCCUPATION_TYPE'] = application_train_df['OCCUPATION_TYPE'].map({'Laborers': 0, 'Core staff': 1, 'Accountants': 2, 'Managers': 3, 'Drivers': 4,
                                                   'Sales staff': 5, 'Cleaning staff': 6, 'Cooking staff': 7, 'Private service staff': 8, 'Medicine staff': 9})                                                                                                     
application_train_df['WEEKDAY_APPR_PROCESS_START'] = application_train_df['WEEKDAY_APPR_PROCESS_START'].map({'MONDAY': 0, 'TUESDAY': 1, 'WEDNESDAY': 2,
                                                                       'THURSDAY': 3, 'FRIDAY': 4, 'SATURDAY': 5, 'SUNDAY': 6})
application_train_df['ORGANIZATION_TYPE'] = application_train_df['ORGANIZATION_TYPE'].astype('category').cat.codes
application_train_df['FONDKAPREMONT_MODE'] = application_train_df['FONDKAPREMONT_MODE'].map({'reg oper account': 0, 'org spec account': 1,
                                                         'not specified': 2, 'org oper account': 3})
application_train_df['HOUSETYPE_MODE'] = application_train_df['HOUSETYPE_MODE'].map({'block of flats': 0, 'specific housing': 1,
                                                 'terraced house': 2, 'specific housing block': 3, np.nan: 4})
application_train_df['WALLSMATERIAL_MODE'] = application_train_df['WALLSMATERIAL_MODE'].map({'Stone, brick': 0, 'Wooden': 1, 'Panel': 2,
                                                         'Block': 3, 'Mixed': 4, 'Monolithic': 5, np.nan: 6})
application_train_df['EMERGENCYSTATE_MODE'] = application_train_df['EMERGENCYSTATE_MODE'].map({'No': 0, 'Yes': 1, np.nan: 2})
print(application_train_df['TARGET'])


NameError: name 'np' is not defined

In [ ]:
application_bureau_df = pd.merge(application_train_df, bureau_df, on = "SK_ID_CURR")
total_overdue = application_bureau_df.groupby("SK_ID_CURR")["AMT_CREDIT_SUM_OVERDUE"].sum()
total_debt = application_bureau_df.groupby("SK_ID_CURR")["AMT_CREDIT_SUM_DEBT"].sum()
times_prolonged = application_bureau_df.groupby("SK_ID_CURR")["CNT_CREDIT_PROLONG"].sum()
days_overdue = application_bureau_df.groupby("SK_ID_CURR")["CREDIT_DAY_OVERDUE"].sum()

In [ ]:
application_train_merged_df = application_train_df.merge(total_overdue, on='SK_ID_CURR', how='left')
application_train_merged_df = application_train_merged_df.merge(total_debt, on='SK_ID_CURR', how='left')
application_train_merged_df = application_train_merged_df.merge(times_prolonged, on='SK_ID_CURR', how='left')
application_train_merged_df = application_train_merged_df.merge(days_overdue, on = "SK_ID_CURR", how = 'left')

In [ ]:
application_train_merged_df['AMT_CREDIT_SUM_OVERDUE'] = application_train_merged_df['AMT_CREDIT_SUM_OVERDUE'].fillna(0)
application_train_merged_df['AMT_CREDIT_SUM_DEBT'] = application_train_merged_df['AMT_CREDIT_SUM_DEBT'].fillna(0)
application_train_merged_df['CNT_CREDIT_PROLONG'] = application_train_merged_df['CNT_CREDIT_PROLONG'].fillna(0)
application_train_merged_df['CREDIT_DAY_OVERDUE'] = application_train_merged_df['CREDIT_DAY_OVERDUE'].fillna(0)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Standardize the four quantitative variables
scaler = StandardScaler()
cols_to_standardize = ['AMT_CREDIT_SUM_OVERDUE', 'AMT_CREDIT_SUM_DEBT', 'CNT_CREDIT_PROLONG', 'CREDIT_DAY_OVERDUE']

application_train_merged_df[cols_to_standardize] = scaler.fit_transform(application_train_merged_df[cols_to_standardize])

print("Standardized columns:")
print(application_train_merged_df[cols_to_standardize].describe())